### 1. Download tracker.tar.gz

In [3]:
%%bash
echo "Downloading tracker.tar.gz...."
gfileid="1CWRj6RzMb7pZ6eiJ2kBaklvXs777dram"
destination_dir="./"
file_name="tracker.tar.gz"
destination_path="${destination_dir}${file_name}"
curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=${gfileid}" > /dev/null
curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=`awk '/download/ {print $NF}' ./cookie`&id=${gfileid}" -o ${destination_path}

echo "Inflating....."
tar -zxf $file_name
rm -rf $file_name cookie

echo "Example: "
ls example

Inflating.....
Example: 
Echabdens_Route_de_la_Gare_5
Echabdens_Route_de_la_Gare_5_cls.npy
Echabdens_Route_de_la_Gare_5_pred
class.names


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   388    0   388    0     0   1874      0 --:--:-- --:--:-- --:--:--  1883
100  172M    0  172M    0     0  2805k      0 --:--:--  0:01:03 --:--:-- 2203k


###  2. Filter out the bounding boxes that lay on the desired areas of the scene

In [2]:
%%bash
python3 gen_valid_annotation.py -h

usage: gen_valid_annotation.py [-h] [--img_w IMG_W] [--img_h IMG_H]
                               [--folder FOLDER] [--seg_path SEG_PATH]
                               [--edge_offset EDGE_OFFSET]
                               [--tolerance_th TOLERANCE_TH]
                               [--save_individually]

optional arguments:
  -h, --help            show this help message and exit
  --img_w IMG_W, -iw IMG_W
                        Image width Default=3480
  --img_h IMG_H, -ih IMG_H
                        Image height Default=2160
  --folder FOLDER, -f FOLDER
                        Folder that contains annotations txt files
  --seg_path SEG_PATH, -sp SEG_PATH
                        Segmentation array in npy format
  --edge_offset EDGE_OFFSET, -e EDGE_OFFSET
                        Ignore bbox which edges are close to bounding of
                        images in pixels Default=25
  --tolerance_th TOLERANCE_TH, -t TOLERANCE_TH
                        Tolerance threshold i.e desir

In [3]:
%%bash
FOLDER="example/Echabdens_Route_de_la_Gare_5_pred/"
SEG_PATH="example/Echabdens_Route_de_la_Gare_5_cls.npy"
python3 gen_valid_annotation.py -f $FOLDER -sp $SEG_PATH


Save result to example/Echabdens_Route_de_la_Gare_5_pred/Echabdens_Route_de_la_Gare_5_all.txt


### 3. Run tracking algorithm

In [4]:
%%bash
python3 tracker.py -h

usage: tracker.py [-h] [--input INPUT] [--img_w IMG_W] [--img_h IMG_H]
                  [--max_distance MAX_DISTANCE] [--max_frame MAX_FRAME]
                  [--start_frame START_FRAME] [--track_cls TRACK_CLS]
                  [--class_infer] [--linear_infer]

optional arguments:
  -h, --help            show this help message and exit
  --input INPUT, -i INPUT
                        Annotation txt file path
  --img_w IMG_W, -iw IMG_W
                        Image width Default=3480
  --img_h IMG_H, -ih IMG_H
                        Image height Default=2160
  --max_distance MAX_DISTANCE, -md MAX_DISTANCE
                        Max distance between consecutive bounding boxes
                        Default=100
  --max_frame MAX_FRAME, -mf MAX_FRAME
                        Max frame to track bounding box Default=20
  --start_frame START_FRAME, -sf START_FRAME
                        Frame number where to start tracking Default=1
  --track_cls TRACK_CLS, -tc TRACK_CLS
              

In [5]:
%%bash
INPUT="example/Echabdens_Route_de_la_Gare_5_pred/Echabdens_Route_de_la_Gare_5_all.txt"
python3 tracker.py -i $INPUT -ci -li

Running Hungarian tracker on example/Echabdens_Route_de_la_Gare_5_pred/Echabdens_Route_de_la_Gare_5_all.txt
Assigning labels....
Merge wrongly separeted trajectories....
Inferring the class in in a trajectory.....
Doing linear interpolation in a trajectory.....
Save result to example/Echabdens_Route_de_la_Gare_5_pred/Echabdens_Route_de_la_Gare_5_tracked_all.txt


### 4. Visaulize the tracking result

In [1]:
%%bash
python3 plot_tracked_bboxes.py -h

usage: plot_tracked_bboxes.py [-h] [--anno_path ANNO_PATH]
                              [--img_path IMG_PATH]
                              [--output_path OUTPUT_PATH]
                              [--class_path CLASS_PATH] [--img_tmp IMG_TMP]
                              [--output_img_tmp OUTPUT_IMG_TMP] [--font FONT]
                              [--font_size FONT_SIZE]
                              [--hide_linear_interpolation]

optional arguments:
  -h, --help            show this help message and exit
  --anno_path ANNO_PATH, -ap ANNO_PATH
                        Annotation txt file path
  --img_path IMG_PATH, -ip IMG_PATH
                        Folder that contains all frames
  --output_path OUTPUT_PATH, -op OUTPUT_PATH
                        Folder that contains output frames
  --class_path CLASS_PATH, -cp CLASS_PATH
                        Class names file
  --img_tmp IMG_TMP, -it IMG_TMP
                        Naming rule of the input images it should be python
          

In [7]:
%%bash
ANNO_PATH="example/Echabdens_Route_de_la_Gare_5_pred/Echabdens_Route_de_la_Gare_5_tracked_all.txt"
IMG_PATH="example/Echabdens_Route_de_la_Gare_5/"
output_path="example/tracked_output/"
CLASS_PATH="example/class.names"
IMG_TMP="Echabdens_Route_de_la_Gare_5_{:06d}.jpg"
OUTPUT_IMG_TMP="Echabdens_Route_de_la_Gare_5_{:06d}.jpg"

python3 plot_tracked_bboxes.py -ap $ANNO_PATH -ip $IMG_PATH -op $output_path -cp $CLASS_PATH -it $IMG_TMP -ot $OUTPUT_IMG_TMP


save plot results to example/tracked_output/Echabdens_Route_de_la_Gare_5_002300.jpg
save plot results to example/tracked_output/Echabdens_Route_de_la_Gare_5_002301.jpg
save plot results to example/tracked_output/Echabdens_Route_de_la_Gare_5_002302.jpg
save plot results to example/tracked_output/Echabdens_Route_de_la_Gare_5_002303.jpg
save plot results to example/tracked_output/Echabdens_Route_de_la_Gare_5_002304.jpg
save plot results to example/tracked_output/Echabdens_Route_de_la_Gare_5_002305.jpg
save plot results to example/tracked_output/Echabdens_Route_de_la_Gare_5_002306.jpg
save plot results to example/tracked_output/Echabdens_Route_de_la_Gare_5_002307.jpg
save plot results to example/tracked_output/Echabdens_Route_de_la_Gare_5_002308.jpg
save plot results to example/tracked_output/Echabdens_Route_de_la_Gare_5_002309.jpg
save plot results to example/tracked_output/Echabdens_Route_de_la_Gare_5_002310.jpg
save plot results to example/tracked_output/Echabdens_Route_de_la_Gare_5_002

In [8]:
%%bash
# Run the cell below if ffmpeg is installed
# Combine images into video and save it to example/output.mp4
ffmpeg -r 25 -pattern_type glob -i "example/tracked_output/*.jpg" example/output.mp4

ffmpeg version 4.0.2 Copyright (c) 2000-2018 the FFmpeg developers
  built with Apple LLVM version 6.1.0 (clang-602.0.53) (based on LLVM 3.6.0svn)
  configuration: --prefix=/usr/local/anaconda3/envs/torch_0.4 --disable-doc --disable-openssl --enable-shared --enable-static --extra-cflags='-Wall -g -m64 -pipe -O3 -march=x86-64 -fPIC' --extra-cxxflags='-Wall -g -m64 -pipe -O3 -march=x86-64 -fPIC' --extra-libs='-lpthread -lm -lz' --enable-zlib --enable-pic --enable-pthreads --enable-gpl --enable-version3 --enable-hardcoded-tables --enable-avresample --enable-libfreetype --enable-gnutls --enable-libx264 --enable-libopenh264
  libavutil      56. 14.100 / 56. 14.100
  libavcodec     58. 18.100 / 58. 18.100
  libavformat    58. 12.100 / 58. 12.100
  libavdevice    58.  3.100 / 58.  3.100
  libavfilter     7. 16.100 /  7. 16.100
  libavresample   4.  0.  0 /  4.  0.  0
  libswscale      5.  1.100 /  5.  1.100
  libswresample   3.  1.100 /  3.  1.100
  libpostproc    55.  1.100 / 55.  1.100
Inpu